In [58]:
import os
import wfdb
import pickle
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.models import load_model
import matplotlib.animation as animation
np.set_printoptions(suppress=True)
%matplotlib inline

In [59]:
win_len = 100
dataset_root = "./dataset_RRI/"
result_root = "./results_max/"
model_root = os.path.join(result_root,"models")
keys = [i.split(".")[0] for i in os.listdir(dataset_root) if "pickle" in i and "cross" not in i ]

In [60]:
def int2label(label):
    if label==0:
        return "Normal"
    elif label==1:
        return "AF"

In [61]:
def get_normalization_param(test_keys):
    max_norm = 0
    min_norm = 10000
    for key in keys:
        if key in test_keys:continue
        with open(os.path.join(dataset_root,key+".pickle"),"rb") as f:
            dataset = pickle.load(f)
        X = dataset["X"]
        if np.max(X) > max_norm:
            max_norm = np.max(X)
        if np.min(X) < min_norm:
            min_norm = np.min(X)
    return min_norm,max_norm

In [62]:
def calc_consistency_once(X,label,seg_idx,model):
    continue_seg = np.concatenate( [X[seg_idx],X[seg_idx+1]],axis=0)
    sliding_win_start = 1
    sliding_win_end = sliding_win_start+win_len
    step = 1

    consistency_list = [] 
    slided_data = []
    while(sliding_win_end<len(continue_seg)):
        data = continue_seg[sliding_win_start:sliding_win_end,:]
        slided_data.append(data)
        sliding_win_start += step
        sliding_win_end += step
        
    slided_data = np.array(slided_data)
    logits = np.argmax(model.predict(slided_data),axis=1)
    labels = [label for i in range(len(logits))]
    consistency_list = np.array(logits==label,dtype=np.float32)
    return consistency_list

In [63]:
def calc_consistency(X,y,start,end,model):
    consistency = []
    consistency_normal = []
    consistency_af = []
    if len(X)==1:
        print("cosistency of can not calculate cosistency")
        return [],[],[]
        
    for seg_idx in tqdm_notebook(range(len(X)-1)):
        seg = X[seg_idx]
        next_seg = X[seg_idx+1]
        seg_end = end[seg_idx]
        next_seg_start = start[seg_idx+1]
        label = np.argmax(y[seg_idx])
        next_label = np.argmax(y[seg_idx+1])
        if not (seg_end == next_seg_start and label==next_label):continue
        consistency_list =  calc_consistency_once(X,label,seg_idx,model)
        consistency.append(consistency_list)
        
        if label == 0:
            consistency_normal.append(consistency_list)
        else:
            consistency_af.append(consistency_list)
        
    assert(len(consistency_normal)+len(consistency_af) == len(consistency))
    #print("cosistency of ",test_key,"=",np.mean(consistency))
    return consistency,consistency_normal,consistency_af

In [64]:
def calc_consistency_for_cross(idx,cross_idx):
    with open(os.path.join(dataset_root,"dataset-cross"+str(cross_idx)+".pickle"),"rb") as f:
        dataset = pickle.load(f)
    test_keys = dataset["test_key"]
    X_test = dataset["X_test"]
    y_test = dataset["y_test"]
    model = load_model(os.path.join(model_root,str(idx)+"-model.h5"))
    min_norm,max_norm = get_normalization_param(test_keys)

    consistencys = []
    consistencys_a = []
    consistencys_n = []
    for test_key in test_keys:
        with open(os.path.join(dataset_root,test_key+".pickle"),"rb") as f:
            dataset = pickle.load(f)
        X = np.expand_dims( (dataset["X"] - min_norm) / (max_norm - min_norm) ,axis=2)
        y = to_categorical (dataset["y"],num_classes=2)
        start = dataset["start"]
        end = dataset["end"]
        consistency,consistency_normal,consistency_af = calc_consistency(X,y,start,end,model)
        consistencys += (consistency)
        consistencys_a += consistency_af
        consistencys_n += consistency_normal

    consistencys = np.concatenate(consistencys,axis=0)
    consistencys_a = np.concatenate(consistencys_a,axis=0)
    consistencys_n = np.concatenate(consistencys_n,axis=0)
    consis = np.mean(consistencys)
    consis_a = np.mean(consistencys_a)
    consis_n = np.mean(consistencys_n)
    return consis,consis_a,consis_n

## Main for Max_pooling

In [9]:
result_path = os.path.join(result_root,"result.csv")
result_df = pd.read_csv(result_path)
result_df = result_df[["idx","cross_idx","repeat_idx","loss","accuracy"]]

In [10]:
consistency = []
consistency_a = []
consistency_n = []
for i in tqdm_notebook(range(len(result_df))):
    row = result_df.iloc[i]
    cross_idx = int(row["cross_idx"])
    idx = int(row["idx"])
    consis,consis_a,consis_n = calc_consistency_for_cross(idx,cross_idx)
    consistency.append(consis)
    consistency_a.append(consis_a)
    consistency_n.append(consis_n)
result_df["consistency"] = consistency
result_df["consistency_af"] = consistency_a
result_df["consistency_normal"] = consistency_n



cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency



In [11]:
result_df["pooling_type"] = ["MaxPooling" for i in range(len(result_df))]
result_df.to_csv(result_path)

## Main for Average_pooling

In [12]:
result_root = "./results_avg/"
model_root = os.path.join(result_root,"models")
result_path = os.path.join(result_root,"result.csv")
result_df = pd.read_csv(result_path)
result_df = result_df[["idx","cross_idx","repeat_idx","loss","accuracy"]]

In [13]:
consistency = []
consistency_a = []
consistency_n = []
for i in tqdm_notebook(range(len(result_df))):
    row = result_df.iloc[i]
    cross_idx = int(row["cross_idx"])
    idx = int(row["idx"])
    consis,consis_a,consis_n = calc_consistency_for_cross(idx,cross_idx)
    consistency.append(consis)
    consistency_a.append(consis_a)
    consistency_n.append(consis_n)
result_df["consistency"] = consistency
result_df["consistency_af"] = consistency_a
result_df["consistency_normal"] = consistency_n


cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency
cosistency of can not calculate cosistency



In [14]:
result_df["pooling_type"] = ["AveragePooling" for i in range(len(result_df))]
result_df.to_csv(result_path)

## Main for BlurMaxpooling

In [72]:
class BlurPooling1D(Layer):
    def __init__(self, filt_size=5, stride=2,**kwargs):
        self.stride = stride
        self.filt_size = filt_size
        self.padding = (int(1.*(filt_size-1)/2), int(np.ceil(1.*(filt_size-1)/2)))
        if(self.filt_size==1):
            self.a = np.array([1.,])
        elif(self.filt_size==2):
            self.a = np.array([1., 1.])
        elif(self.filt_size==3):
            self.a = np.array([1., 2., 1.])
        elif(self.filt_size==4):    
            self.a = np.array([1., 3., 3., 1.])
        elif(self.filt_size==5):    
            self.a = np.array([1., 4., 6., 4., 1.])
        elif(self.filt_size==6):    
            self.a = np.array([1., 5., 10., 10., 5., 1.])
        elif(self.filt_size==7):    
            self.a = np.a
            rray([1., 6., 15., 20., 15., 6., 1.])
        super(BlurPooling1D, self).__init__(**kwargs)
        
    def compute_output_shape(self, input_shape):
        win_len = input_shape[1] // self.stride
        channels = input_shape[2]
        return (input_shape[0], win_len, channels)
        
    def call(self, x):
        k = self.a
        k = k / np.sum(k)
        k = np.tile(k[:,None,None], (1,x.shape[-1],1) )                
        k = K.constant (k, dtype=K.floatx() )

        x = K.temporal_padding(x, padding=self.padding)
        x = K.conv1d(x,k,strides=self.stride,padding='valid')
        
        #x = MaxPooling1D(pool_size=2,strides=2,padding="same")(x)
        return x

In [77]:
result_root = "./results_maxblur-1/"
model_root = os.path.join(result_root,"models")